# RAG 평가 개요
- RAG 평가란 RAG 시스템이 주어진 입력에 대해 얼마나 효과적으로 관련 정보를 검색하고, 이를 기반으로 정확하고 유의미한 응답을 생성하는지를 측정하는 과정이다. 
- **평가 요소**
    - **검색 단계 평가**
        - 입력 질문에 대해 검색된 문서나 정보의 관련성과 정확성을 평가.
    - **생성 단계 평가**
        - 검색된 정보를 기반으로 생성된 응답의 품질, 정확성등을 평가.
- **평가 방법**
    - 온/오프라인 평가
        1. **오프라인 평가**
            - 미리 준비된 데이터셋을 활용하여 RAG 시스템의 성능을 측정한다.
        2. **온라인 평가**
            - 실제 사용자 트래픽과 피드백을 기반으로 시스템의 실시간 성능을 평가한다.
    - 정량적/정성적 평가
        1. 정량적 평가
            - 자동화된 지표를 사용하여 생성된 텍스트의 품질을 평가한다.
        2. 정성적 평가
            - 전문가나 일반 사용자가 직접 생성된 응답의 품질을 평가하여 주관적인 지표를 평가한다.

# [RAGAS](https://www.ragas.io/)
- RAGAS는 RAG 파이프라인을 **정량적 으로 평가하는** 오픈소스 프레임 워크이다. 
- RAGAS 문서: https://docs.ragas.io/en/stable/
## 설치
- `pip install ragas`

## RAGAS 평가 지표 개요
![ragas_score](figures/ragas_score.png)
- **Generation**
    - llm 모델이 생성한 답변에 대한 평가 지표들.
    - **Faithfulness(신뢰성)**
        -  생성된 답변과 검색된 문서(context)간의 관련성을 평가하는 지표
        -  생성된 답변이 주어진 문맥(context)에 얼마나 충실한지를 평가하는 지표로 할루시네이션에 대한 평가로 볼 수있다.
    - **Answer relevancy(답변 적합성)**
        - 생성된 답변과 사용자의 질문간의 관련성을 평가하는 지표
        - 생성된 답변이 사용자의 질문과 얼마나 관련성이 있는지를 평가하는 지표.
- **Retrieval**
    -  질문에 대해 검색한 문서(context)들에 대한 평가
    -  **Context Precision(문맥 정밀도)**
        -  검색된 문서(context)들 중 질문과 관련 있는 것들이 **얼마나 상위 순위에 위치하는지** 평가하는 지표.
    -  **Context Recall(문맥 재현률)**
        -  검색된 문서(context)가 정답(ground-truth)의 정보를 얼마나 포함하고 있는지 평가하는 지표.
- 이러한 지표들은 RAG 파이프라인의 성능을 다각도로 평가하는 데 활용된다.
![RAGAS_score2](figures/RAGAS_score2.png)

## 주요 평가지표
### Generation 평가
- LLM이 생성한 답변에 대한 평가
  
#### Faithfulness (신뢰성)
- 생성된 답변이 얼마나 주어진 검색 문서들(context)를 잘 반영해서 생성되었는지 평가한다. 할루시네이션에 대한 평가라고 할 수 있다. 
- 점수범위: **0 ~ 1** (1에 가까울수록 좋음)
- 답변에 포함된 모든 주장이 context에서 얼마나 추출 가능한지를 확인한다.

##### 평가 방법
1. Answer에서 주장 구문(claim statement)들을 생성(추출)한다. (주장이란, 질문(user input)과 관련된 내용)
    - 예) 
        - **질문**: 한국의 수도는 어디이고 인구는 얼마나 되나요? 
        - **LLM 답변**: 한국의 수도는 서울이고 인구수는 3000만명이다. 
        - **주장(claim)**: 
            1. 한국의 수도는 서울이다.
            2. 인구수는 3000만명이다.
2. 각 주장들을 context로 부터 추론 가능한지 판단한다. 이를 바탕으로 faithfulness 점수를 계산한다.
    - 예)
        - context: 한국은 동아시아에 위치하고 있는 나라다. 한국의 수도는 서울이다. .... 한국의 인구는 5000만명이고 서울에 1000만이 살고 있다.
        - 위 context에서 추론 가능한 주장: 
            - 한국의 수도는 서울이다. -> context에서 추론가능한 주장.
            - 한국의 인구는 3000만명이다. -> context에서 추론 불가능한 주장.
3. **Faithfulness score** 를 계산한다. 총 주장 수 중에서 context로 부터 추론가능한 주장의 개수.    
    - 예)
        - Faithfulness Score = $\cfrac{1}{2} = 0.5$ (두 개의 주장 중 한 개의 주장만 context에서 유추할 수있다.)
    - LLM 답변에서 주장을 추출 하는 것과 각 주장이 context에서 추론 가능한 지를 판단하는 것은 LLM 을 활용한다.
- 공식
    $$
    \text{Faithfulness Score}\;=\;\cfrac{\text{주어진\;context\;에서\;추론할\;수\;있는\;주장의\;개수}}{\text{총\;주장\;개수}}
    $$

#### Answer relevancy (답변 적합성)
- 생성된 답변이 질문(user input)에 얼마나 잘 부합하는 지를 평가한다.
- 점수 범위: -1~1 (1에 가까울수록 좋음)
- LLM이 생성한 답변을 기반으로 질문들을 생성한다. 이렇게 생성한 질문들과 실제 질문(user input)의 embedding vector 간의 **코사인 유사도**를 측정한다.

##### 평가방법
1. LLM이 생성한 답변을 기반으로 질문들을 생성한다.
    - 예) 
        - **LLM** 답변: 한국의 수도는 서울이고 인구수는 3000만명이다. 
        - **생성된 질문**: 
            1. 한국의 수도는 어디이고 인구는 얼마나 되나요?
            2. 한국의 수도는 어디인가요?
            3. 한국의 인구는 얼마나 되나요?
2. 실제 질문과 생성한 질문간의 코사인 유사도를 측정한다. 그 평균이 최종 점수가 된다.
    - 예)
        - **실제 질문**: 한국의 수도는 어디이고 인구는 얼마나 되나요?
        - **생성된 질문**: 
            1. 한국의 수도는 어디이고 인구는 얼마나 되나요?
            2. 한국의 수도는 어디인가요?
            3. 한국의 인구는 얼마나 되나요?
- 공식
  $$
    \cfrac{1}{N} \sum_{i=1}^{N} \text{cosine\_similarity}(q_{\text{user}_{_i}}, q_{\text{generated}})
  $$

### Retrieval 평가
User input에 대해 Vector store에서 검색한 context에 대한 평가

#### Context Precision
- 검색된 문서(context)들 중 **질문과 관련 있는 것들이 얼마나 상위 순위**에 있는 지 평가.
- 점수 범위: 0~1 (1에 가까울수록 좋음)


##### 평가방법

- 공식
$$
 \text{Context\;Precision@K} = \frac{\sum_{k=1}^{K} \left( \text{Precision@k} \times v_k \right)}{\ 상위\;K개\;결과에서의\;관련\;항목\;수}
$$
$$
 \text{Precision@k} = \frac{\text{True\;positive@k}}{(\text{True\;positive@k} + \text{False\;positive@k})} \\
$$
- $\text{Precision@k}$: 개별 문서에 대한 Precision
- K: context 의 개수(chuck 수)
- $v_k$: 질문과의 context간의 관련성 여부로 0 또는 1. (0: 관련 없음, 1: 관련 있음)

##### 예시
- 질문과 context 관련성 예
    - 질문: 한국의 수도는 어디이고 인구는 얼마나 되나요?
    - 높은 정밀도 context
        - 한국의 수도는 서울이고 인구는 5000만명 입니다. 
        - 한국의 수도는 서울입니다.
        - 한국은 동아시아에 위치해 있는 국가로 수도는 서울입니다.
        - 한국의 인구는 5000만명 입니다.
    - 낮은 정밀도 context
        - 한국은 동아시아에 위치한 국가입니다.
        - 한국의 K-pop은 전 세계적으로 유명합니다.
        - 비빔밥, 불고기는 한국의 대표적인 음식입니다.
    - **높은 정밀도의 context이 상위 순위에 위치했으면 높은 점수를 받는다.**
- 상위 5개의 검색 결과 중 1번째, 3번째, 4번째 문서가 관련이 있다고 가정
    ```bash
    Precision@1 = 1/1 = 1.0    # True positive@1/(True positive@1 + False positive@1).  1/1(1번 문서 계산 시에는 1개 문서만 있으므로 분모가 1이 된다.)
    Precision@2 = 1/2 = 0.5     
    Precision@3 = 2/3 ≈ 0.67    
    Precision@4 = 3/4 = 0.75
    Precision@5 = 3/5 = 0.6
    ```
- vk의 값
    - 1번째: $v_1 = 1$
    - 2번째: $v_2 = 0$
    - 3번째: $v_3 = 1$
    - 4번째: $v_4 = 1$
    - 5번째: $v_5 = 0$

- Context Precision@5
$$
\text{Context\;Precision@5} = \frac{(1.0 \times 1) + (0.5 \times 0) + (0.67 \times 1) + (0.75 \times 1) + (0.6 \times 0)}{3} = \frac{1.0 + 0 + 0.67 + 0.75 + 0}{3} ≈ 0.807
$$

#### Context Recall (문맥 재현률)
- 검색된 문서(context)가 얼마나 정답(ground-truth)의 정보를 포함있는 지 평가하는 지표
- 점수 범위: 0~1 (1에 가까울수록 좋음)
- **정답(ground truth)의 각 주장(claim)이 검색된 context와 얼마나 일치**하는지 계산함.

##### 평가방법
1. **정답**에서 주장 문장(claim statement)들을 생성(추출)한다.
    - 예) 
        - **정답**: 한국의 수도는 서울이고 인구수는 5000만명이다. 
        - **주장(claim)**: 
            1. 한국의 수도는 서울이다.
            2. 인구수는 5000만명이다.
2. 각 주장 문장(claim statement)의 정보를 검색된 context들에서 찾을 수 있는지 판별한다. 이를 바탕으로 context recall 점수를 계산한다.
    - 예)
        - context: 한국은 동아시아에 위치하고 있는 나라다. 한국의 수도는 서울이다.
        - 위 context에서 추론 가능한 주장: 
            - 한국의 수도는 서울이다. -> context에서 찾을 수 있다.
            - 한국의 인구는 5000만명이다. -> context에서 찾을 수 없다.
3. **Context Recall Score** 를 계산한다. 총 주장 수 중에서 context로 부터 찾을 수 있는 주장의 개수.
    - 예)
        - Context Recall Score = $\cfrac{1}{2} = 0.5$ (두 개의 주장 중 한 개의 주장만 context에서 찾을 수 있다.)

- 공식
    $$
    \text{Context Recall Score}\;=\;\cfrac{\text{GT의\;주장\;중\;주어진\;context\;에서\;찾을\;수\;있는\;주장의\;개수}}{\text{GT의\;총\;주장\;개수}}
    $$ 

# RAGAS 평가 실습

In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(model="gpt-4o-mini") #평가시 사용할 llm 모델은 성능 좋은 것을 써야 좀 더 정확한 평가가 가능.
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small"
)


## RAG Chain 구성
- Vector Store 연결
- Chain 응답 결과
    - 평가를 위해 **Vector Store에서 검색한 context**들과 **LLM 응답**이 출력되도록 chain을 구성한다.

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

COLLECTION_NAME = "olympic_info"
PERSIST_DIRECTORY = "vector_store/olympic_info"

loader = TextLoader("data/olympic.txt", encoding='utf-8')
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini",
    chunk_size=500,
    chunk_overlap=100
)
docs = loader.load_and_split(splitter)

vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

In [2]:
from langchain_chroma import Chroma
# 연결
COLLECTION_NAME = "olympic_info"
PERSIST_DIRECTORY = "vector_store/olympic_info"

vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)

In [3]:
vector_store._collection.count()

36

In [9]:
from operator import itemgetter
# 주어진 key/index의 값(원소)를 자료구조에서 추출해주는 함수.
d = {"name":"홍길동", "age":30}
i_getter = itemgetter("name")  # name key의 value를 추출
i_getter(d)


'홍길동'

In [16]:
# RAG 체인 구성
# query -> pt -> llm -> 응답 (query에 같이 입력된 context, 답변 => 2가지를 출력)
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from operator import itemgetter

# prompt template. langchain hub에 등록된 것을 가져와서 사용.
prompt_template = hub.pull("rlm/rag-prompt")
# prompt_template

# Retriever 생성
retriever = vector_store.as_retriever()


def format_docs(src_docs:dict[str, list[Document]]) -> str:
    """list[Document]: Vector Store에서 검색한 context들에서 
    page_content만 추출해서 하나의 문자열로 합쳐서 반환"""
    docs = src_docs['context']
    return "\n\n".join([doc.page_content for doc in docs])

def str_from_documents(docs: list[Document]) -> list[str]:
    """list[Document]에서 page_content 값들만 추출한 list를 반환."""
    return [doc.page_content for doc in docs]

rag_chain = (
    RunnablePassthrough() # rag chain을 RunnableSequence로 만들기 위해 Runnable인 것으로 시작.
    | {
        "context": retriever, "question":RunnablePassthrough()
    } # retriver -> {"context":list[Document], "question":"user input"}
    | {
        # 앞에서 넘어온 dictionary에서 context(List[Document])를 추출 -> page_content값들을 list로 반환. list[str]
        "source_context" : itemgetter("context") | RunnableLambda(str_from_documents), 
        "llm_answer": {
            # {"context":list[Document]} -> str(page_content들만 모은 string)
            "context": RunnableLambda(format_docs), "question":itemgetter("question")
        } | prompt_template | model | StrOutputParser()  # LLM 응답 처리 chain. 
    }
)

In [17]:
user_input = "국제 올림픽 위원회에 대해서 설명해 주세요."
response = rag_chain.invoke(user_input)
# response: dictionary - {source_context:VectorStore에 조회한 context들, llm_answer:LLM 답변변}

In [18]:
print(type(response))
print(response.keys())
print("답변:", response['llm_answer'])
response['source_context']

<class 'dict'>
dict_keys(['source_context', 'llm_answer'])
답변: 국제 올림픽 위원회(IOC)는 올림픽 활동을 총괄하는 기구로, 올림픽 개최 도시 선정, 종목 변경, 스폰서 계약 등을 관리합니다. IOC는 국제경기연맹(IF), 국가 올림픽 위원회(NOC), 올림픽 조직 위원회(OCOG)로 구성되며, 올림픽의 공식언어는 프랑스어와 영어 및 개최국의 공용어입니다. IOC는 변화하는 사회 환경에 적응하며 올림픽의 상업화와 다양성을 증진시키기 위해 노력하고 있습니다.


['국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.\n- 국제경기연맹(IF)은 국제적인 규모의 경기를 관리, 감독하는 기구이다. 예를 들어서 국제 축구 연맹(FIFA)는 축구를 주관하며, 국제 배구 연맹(FIVB)은 배구를 주관하는 기구이다. 올림픽에는 현재 35개의 국제경기연맹이 있고 각 종목을 대표한다. (이 중에는 올림픽 종목은 아니지만 IOC의 승인을 받은 연맹도 있다.)\n- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.\n- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원회는 올림픽이 끝나면 해산되며 최종보고서를 IOC에 제출한다.\n올림픽의 공식언어는 프랑스어와 영어와 개최국의 공용어이다. 모든 선언(예를 들어서 개막식 때 각국 소개를 할 때)들은 세 언어가 모두 나오거나 영어나 프랑스어 중에서 한 언어로만 말하기도 한다. 개최국의 공용어가 영어나 프랑스어가 아닐 때는 당연히 그 나라의 공용어도 함께 나온다.',
 '또한 20세기에 올림픽 운동이 발전함에 따라, IOC는 변화하는 세계의 사회 환경에 적응해야 했다. 이러한 변화의 예로는 얼음과 눈을 이용한 경기 종목을 다루는 동계 올림픽, 장애인이 참여하는 패럴림픽, 스페셜 올림픽, 데플림픽, 10대 선수들이 참여하는 유스 올림픽 등을 들 수 있다. 그 뿐만 아니라 IOC는 20세기의 변화하는 경제, 정치, 기

## 평가

In [22]:
user_input = "국제 올림픽 위원회에 대해 설명해주세요."
answer = response['llm_answer']
context_list = response['source_context']
ground_truth = "국제 올림픽 위원회(IOC)는 올림픽 활동을 총괄하는 기구로, 올림픽 개최 도시 선정, 종목 변경, 스폰서 계약 등을 관리합니다."
print(user_input) # 질문
print(answer)     # llm 모델 답변
print(context_list) # vector store에서 조회한 context들.
print(ground_truth) # 정답

국제 올림픽 위원회에 대해 설명해주세요.
국제 올림픽 위원회(IOC)는 올림픽 활동을 총괄하는 기구로, 올림픽 개최 도시 선정, 종목 변경, 스폰서 계약 등을 관리합니다. IOC는 국제경기연맹(IF), 국가 올림픽 위원회(NOC), 올림픽 조직 위원회(OCOG)로 구성되며, 올림픽의 공식언어는 프랑스어와 영어 및 개최국의 공용어입니다. IOC는 변화하는 사회 환경에 적응하며 올림픽의 상업화와 다양성을 증진시키기 위해 노력하고 있습니다.
['국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.\n- 국제경기연맹(IF)은 국제적인 규모의 경기를 관리, 감독하는 기구이다. 예를 들어서 국제 축구 연맹(FIFA)는 축구를 주관하며, 국제 배구 연맹(FIVB)은 배구를 주관하는 기구이다. 올림픽에는 현재 35개의 국제경기연맹이 있고 각 종목을 대표한다. (이 중에는 올림픽 종목은 아니지만 IOC의 승인을 받은 연맹도 있다.)\n- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.\n- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원회는 올림픽이 끝나면 해산되며 최종보고서를 IOC에 제출한다.\n올림픽의 공식언어는 프랑스어와 영어와 개최국의 공용어이다. 모든 선언(예를 들어서 개막식 때 각국 소개를 할 때)들은 세 언어가 모두 나오거나 영어나 프랑스어 중에서 한 언어로만 말하기도 한다. 개최

In [24]:
from ragas import SingleTurnSample, EvaluationDataset
# 평가 데이터셋을 구성.

## 개별 평가 데이터
sample = SingleTurnSample(
    user_input=user_input,           # 사용자 입력-질문
    retrieved_contexts=context_list, # 질문에 대해서 Vector Store에서 조회한 context들.
    response=answer,                 # LLM 답변 (정답 추정값)
    reference=ground_truth           # 정답 답변
)

# 평가 데이터셋을 구성
eval_dataset = EvaluationDataset(samples=[sample])
print(type(eval_dataset))
eval_dataset

<class 'ragas.dataset_schema.EvaluationDataset'>


EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=1)

In [25]:
eval_dataset[0]

SingleTurnSample(user_input='국제 올림픽 위원회에 대해 설명해주세요.', retrieved_contexts=['국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 • 미디어 파트너를 맺기 • 선수, 직원, 심판, 모든 사람과 기관이 올림픽 헌장을 지키는 것을 말한다. 국제올림픽위원회(IOC)는 모든 올림픽 활동을 통솔하는 단체로서, 올림픽 개최 도시 선정, 계획 감독, 종목 변경, 스폰서 및 방송권 계약 체결 등의 권리가 있다. 올림픽 활동은 크게 세 가지로 구성된다.\n- 국제경기연맹(IF)은 국제적인 규모의 경기를 관리, 감독하는 기구이다. 예를 들어서 국제 축구 연맹(FIFA)는 축구를 주관하며, 국제 배구 연맹(FIVB)은 배구를 주관하는 기구이다. 올림픽에는 현재 35개의 국제경기연맹이 있고 각 종목을 대표한다. (이 중에는 올림픽 종목은 아니지만 IOC의 승인을 받은 연맹도 있다.)\n- 국가 올림픽 위원회(NOC)는 각국의 올림픽 활동을 감독하는 기구이다. 예를 들어서 대한 올림픽 위원회(KOC)는 대한민국의 국가 올림픽 위원회이다. 현재 IOC에 소속된 국가 올림픽 위원회는 205개이다.\n- 올림픽 조직 위원회(OCOG)는 임시적인 조직으로 올림픽의 총체적인 것(개막식, 페막식 등)을 책임지기 위해 구성된 조직이다. 올림픽 조직 위원회는 올림픽이 끝나면 해산되며 최종보고서를 IOC에 제출한다.\n올림픽의 공식언어는 프랑스어와 영어와 개최국의 공용어이다. 모든 선언(예를 들어서 개막식 때 각국 소개를 할 때)들은 세 언어가 모두 나오거나 영어나 프랑스어 중에서 한 언어로만 말하기도 한다. 개최국의 공용어가 영어나 프랑스어가 아닐 때는 당연히 그 나라의 공용어도 함께 나온다.', '또한 20세기에 올림픽 운동이 발전함에 따라, IOC는 변화하는 세계의 사회 환경에 적응해야 했다. 이러한 변화의 예로는 얼음과 눈을 이용한 경기 종목을 다루는 동계 올림픽, 장애인이 참여하는 패럴림픽, 스페셜 올림픽, 

In [26]:
# Dataset을 Pandas DataFrame으로 변환
eval_df = eval_dataset.to_pandas()
eval_df

,user_input,retrieved_contexts,response,reference
0,국제 올림픽 위원회에 대해 설명해주세요.,"[국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 •...","국제 올림픽 위원회(IOC)는 올림픽 활동을 총괄하는 기구로, 올림픽 개최 도시 선...","국제 올림픽 위원회(IOC)는 올림픽 활동을 총괄하는 기구로, 올림픽 개최 도시 선..."


In [30]:
# 평가

from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas import evaluate

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(model="gpt-4o") # 평가시 사용할 llm 모델.
eval_llm = LangchainLLMWrapper(model) # Langchain 모델을 ragas에서 사용할 수있도록 변환.

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)

In [32]:
#평가 지표들을 List로 묶는다.
metrics = [
    LLMContextRecall(llm=eval_llm),
    LLMContextPrecisionWithReference(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)
]
result = evaluate(dataset=eval_dataset, metrics=metrics)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [35]:
type(result)
result

{'context_recall': 1.0000, 'llm_context_precision_with_reference': 1.0000, 'faithfulness': 0.5385, 'answer_relevancy': 0.4782}

In [ ]:
# context_recall: context와 ground-truth 간의 관련성
# llm_context_precision_with_reference: context와 user input(사용자 질문)과의 관련성성
# faithfulness: 모델이 출력한 정답예측과 context간의 관련성.
# answer_relevancy: 모델이 출력한 정답예측과 사용자 질문간의 관계

In [36]:
result.to_pandas()

,user_input,retrieved_contexts,response,reference,context_recall,llm_context_precision_with_reference,faithfulness,answer_relevancy
0,국제 올림픽 위원회에 대해 설명해주세요.,"[국제 올림픽 위원회\n올림픽 활동이란 많은 수의 국가, 국제 경기 연맹과 협회 •...","국제 올림픽 위원회(IOC)는 올림픽 활동을 총괄하는 기구로, 올림픽 개최 도시 선...","국제 올림픽 위원회(IOC)는 올림픽 활동을 총괄하는 기구로, 올림픽 개최 도시 선...",1.0,1.0,0.538462,0.478239


# 평가 데이터 셋 만들기

1. 문서를 Loading하고 split하여 Context 들을 만든다.
2. Context들 중 평가에 사용할 것을 Random하게 선택한다.
3. 선택된 context를 기반으로 LLM 모델을 이용해서 질문과 답변을 생성한다. 
4. 생성된 질문과 답변을 검토하여 품질을 높인다. 
   - 질문과 답변 자체가 맞는지 검사한다.
   - 나올만한 질문인 지 확인한다.

- **생성된 질문-답변의 질이 낮으면 좋은 평가를 할 수 없다.**
    - 질문-답변 생성시 사용하는 LLM 모델은 성능이 좋은 것을 사용해야 한다. 
    - 생성된 질문-답변을 사람이 검토해서 품질을 더 높여야 한다.

## 데이터셋에 포함될 내용
- Dataset을 생성할 때는 다음 항목이 들어가야 한다.
    -  user_input: 질문. `string`
    -  reference : 정답. `string`
    -  retrieved_contexts : context. `list(string)`
-  평가할 때 추가할 것
    - response: LLM 모델이 생성한 답변. `string`

In [38]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("data/olympic.txt", encoding='utf-8')
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4o-mini",
    chunk_size=500,
    chunk_overlap=100
)
docs = loader.load_and_split(splitter)
print(len(docs))

36


In [42]:
import random
l = list(range(36))
random.shuffle(l)
l[:5]

[14, 28, 9, 2, 10]

In [44]:
# 평가 데이터로 사용할 context k개를 랜덤하게 추출
import random
total_samples = 5 # 추출할 샘플 개수
eval_context_list = [] # Sample들을 담을 리스트
while len(eval_context_list) < 5:
    _context = docs[random.randint(0, len(docs)-1)].page_content
    if len(_context) < 100: # 글자수 너무 적으면 질문을 생성 못하기 때문에 글자수 체크.
        continue
    eval_context_list.append(_context)

In [45]:
len(eval_context_list)

5

In [69]:
# Chain을 이용해서 LLM에게 질문-답 생성을 요청.
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from textwrap import dedent

# JSONOutputParser에서 사용할 스키마 생성.
class EvalDatasetSchema(BaseModel):
    user_input:str = Field(..., description="질문(Question)")
    retrieved_contexts:list[str] = Field(..., description="LLM이 답변할 때 참조할 context")
    reference: str = Field(..., description="정답(ground truth)")

parser = JsonOutputParser(pydantic_object=EvalDatasetSchema)
# print(parser.get_format_instructions())

prompt_template = PromptTemplate.from_template(
    template=dedent("""
        당신은 RAG 평가를 위해 질문과 정답 쌍을 생성하는 인공지능 비서입니다.
        다음 [Context] 에 문서가 주어지면 해당 문서를 기반으로 {num_questions}개의 질문을 생성하세요. 

        질문과 정답을 생성한 후 아래의 출력 형식 GUIDE 에 맞게 생성합니다.
        질문은 반드시 [context] 문서에 있는 정보를 바탕으로 생성해야 합니다. [context]에 없는 내용을 가지고 질문-답변을 절대 만들면 안됩니다.
        질문은 간결하게 작성합니다.
        하나의 질문에는 한 가지씩만 내용만 작성합니다. 
        질문을 만들 때 "제공된 문맥에서", "문서에 설명된 대로", "주어진 문서에 따라" 또는 이와 유사한 말을 하지 마세요.
        정답은 반드시 [context]에 있는 정보를 바탕으로 작성합니다. 없는 내용을 추가하지 않습니다.
        질문과 답변을 만들고 그 내용이 [context] 에 있는 항목인지 다시 한번 확인합니다.
        생성된 질문-답변 쌍은 반드시 dictionary 형태로 정의하고 list로 묶어서 반환해야 합니다.
        질문-답변 쌍은 반드시 {num_questions}개를 만들어 주십시오.
                    
        출력 형식: {format_instructions}

        [Context]
        {context}
        """
    ),
    partial_variables={"format_instructions":parser.get_format_instructions()}
)

model = ChatOpenAI(model="gpt-4o")
dataset_generator_chain = prompt_template | model | parser

In [70]:
c = eval_context_list[0]

qa = dataset_generator_chain.invoke({"context":c, "num_questions":5})

In [71]:
len(qa)

5

In [60]:
# 확인 후 retrieved_contexts을 context로 변경.
for d in qa:
    d['retrieved_contexts'] = [c]
qa

[{'user_input': '세계반도핑기구(WADA)는 언제 설립되었나요?',
  'retrieved_contexts': ['1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 세계반도핑기구(WADA)를 설립한다. 2000년 하계 올림픽과 2002년 동계 올림픽 때는 약물 양성 반응을 보인 선수들이 급격히 증가했고, 역도와 크로스컨트리에서는 몇몇 선수들이 도핑 테스트에 걸려서 실격되기도 했다. 2006년 동계 올림픽 때는 메달리스트 한 명이 양성반응을 보여 메달을 반납해야 했다. IOC가 만든 약물 반응 판정(현재 올림픽 도핑테스트의 기준이 됨)은 인정을 받게 되었고 이제는 다른 경기 연맹에서도 벤치마킹을 할 정도가 되었다. 2008년 베이징 올림픽 기간중에는 3,667명의 선수들이 세계반도핑기구의 검사를 받았으며 소변과 혈액 검사로 약물 복용 검사를 했다. 몇몇 선수들은 국가 올림픽 위원회(NOC)에 의해 올림픽이 시작되기 전에 출전금지 조치를 당했고, 올림픽 기간중에는 단 3명만이 도핑 검사에 걸렸다.'],
  'reference': '세계반도핑기구(WADA)는 1999년에 설립되었습니다.'},
 {'user_input': '2006년 동계 올림픽에서 어떤 일이 있었나요?',
  'retrieved_contexts': ['1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 세계반도핑기구(WADA)를 설립한다. 2000년 하계 올림픽과 2002년 동계 올림픽 때는 약물 양성 반응을 보인 선수들이 급격히 증가했고, 역도와 크로스컨트리에서는 몇몇 선수들이 도핑 테스트에 걸려서 실격되기도 했다. 2006년 동계 올림픽 때는 메달리스트 한 명이 양성반응을 보여 메달을 반납해야 했다. IOC가 만든 약물 반응 판정(현재 올림픽 도핑테스트의 기준이 됨)은 인정을 받게 되었고 이제는 다른 경기 연맹에서도 벤치마킹을 할 정도가 되었다. 2008년 베이징 올림픽 기간중에는 3,667명의 선수들이 세계반도핑기구의 검사

In [61]:
import pandas as pd
pd.DataFrame(qa)

,user_input,retrieved_contexts,reference
0,세계반도핑기구(WADA)는 언제 설립되었나요?,"[1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 ...",세계반도핑기구(WADA)는 1999년에 설립되었습니다.
1,2006년 동계 올림픽에서 어떤 일이 있었나요?,"[1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 ...",2006년 동계 올림픽에서는 메달리스트 한 명이 양성반응을 보여 메달을 반납해야 했...
2,올림픽 도핑 테스트의 기준은 무엇인가요?,"[1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 ...",IOC가 만든 약물 반응 판정이 올림픽 도핑 테스트의 기준입니다.
3,2008년 베이징 올림픽에서는 몇 명의 선수가 도핑 검사를 받았나요?,"[1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 ...","2008년 베이징 올림픽에서는 3,667명의 선수가 도핑 검사를 받았습니다."
4,2008년 베이징 올림픽에서 도핑 검사에 걸린 선수는 몇 명인가요?,"[1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 ...",2008년 베이징 올림픽 기간 중에는 단 3명의 선수가 도핑 검사에 걸렸습니다.


In [73]:
### 전체 context sample들로 qa dataset을 생성
eval_data_list = []
num_questions = 5  # context 당 k(5) 개 QA 쌍 생성.
for context in eval_context_list:
    _eval_data_list = dataset_generator_chain.invoke(
        {"context":context, "num_questions":num_questions}
    )
    # retrieve_contexts의 값을 변경
    # print(_eval_data_list)
    for eval_data in _eval_data_list:
        eval_data['retrieved_contexts'] = [context]
    
    eval_data_list.extend(_eval_data_list)

In [76]:
len(eval_data_list)
eval_data_list[:2]

[{'user_input': '세계반도핑기구(WADA)는 언제 설립되었나요?',
  'retrieved_contexts': ['1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 세계반도핑기구(WADA)를 설립한다. 2000년 하계 올림픽과 2002년 동계 올림픽 때는 약물 양성 반응을 보인 선수들이 급격히 증가했고, 역도와 크로스컨트리에서는 몇몇 선수들이 도핑 테스트에 걸려서 실격되기도 했다. 2006년 동계 올림픽 때는 메달리스트 한 명이 양성반응을 보여 메달을 반납해야 했다. IOC가 만든 약물 반응 판정(현재 올림픽 도핑테스트의 기준이 됨)은 인정을 받게 되었고 이제는 다른 경기 연맹에서도 벤치마킹을 할 정도가 되었다. 2008년 베이징 올림픽 기간중에는 3,667명의 선수들이 세계반도핑기구의 검사를 받았으며 소변과 혈액 검사로 약물 복용 검사를 했다. 몇몇 선수들은 국가 올림픽 위원회(NOC)에 의해 올림픽이 시작되기 전에 출전금지 조치를 당했고, 올림픽 기간중에는 단 3명만이 도핑 검사에 걸렸다.'],
  'reference': '세계반도핑기구(WADA)는 1999년에 설립되었습니다.'},
 {'user_input': '2006년 동계 올림픽에서 어떤 일이 있었나요?',
  'retrieved_contexts': ['1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 세계반도핑기구(WADA)를 설립한다. 2000년 하계 올림픽과 2002년 동계 올림픽 때는 약물 양성 반응을 보인 선수들이 급격히 증가했고, 역도와 크로스컨트리에서는 몇몇 선수들이 도핑 테스트에 걸려서 실격되기도 했다. 2006년 동계 올림픽 때는 메달리스트 한 명이 양성반응을 보여 메달을 반납해야 했다. IOC가 만든 약물 반응 판정(현재 올림픽 도핑테스트의 기준이 됨)은 인정을 받게 되었고 이제는 다른 경기 연맹에서도 벤치마킹을 할 정도가 되었다. 2008년 베이징 올림픽 기간중에는 3,667명의 선수들이 세계반도핑기구의 검사

In [77]:
eval_df = pd.DataFrame(eval_data_list)
eval_df.shape

(25, 3)

In [78]:
eval_df.head()

,user_input,retrieved_contexts,reference
0,세계반도핑기구(WADA)는 언제 설립되었나요?,"[1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 ...",세계반도핑기구(WADA)는 1999년에 설립되었습니다.
1,2006년 동계 올림픽에서 어떤 일이 있었나요?,"[1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 ...",2006년 동계 올림픽 때 메달리스트 한 명이 양성반응을 보여 메달을 반납했습니다.
2,2008년 베이징 올림픽 기간 동안 몇 명의 선수가 도핑 검사를 받았나요?,"[1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 ...","2008년 베이징 올림픽 기간 동안 3,667명의 선수가 도핑 검사를 받았습니다."
3,IOC가 만든 약물 반응 판정은 어떤 역할을 했나요?,"[1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 ...","IOC가 만든 약물 반응 판정은 올림픽 도핑 테스트의 기준이 되었고, 다른 경기 연..."
4,2000년 하계 올림픽과 2002년 동계 올림픽에서 어떤 도핑 문제들이 발생했나요?,"[1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 ...",2000년 하계 올림픽과 2002년 동계 올림픽에서는 약물 양성 반응을 보인 선수들...


### HuggingFace hub에 생성된 평가 Dataset 저장
- 생성한 평가 데이터셋을 HuggingFace hub에 저장하면 datasets을 이용해 load해서 사용 할 수 있다.

In [79]:
from datasets import Dataset
import huggingface_hub as hub

# DataFrame으로 부터 Dataset을 생성
eval_dataset = Dataset.from_pandas(eval_df)
eval_dataset

Dataset({
    features: ['user_input', 'retrieved_contexts', 'reference'],
    num_rows: 25
})

In [81]:
# 로그인
hub.login()

In [82]:
# dataset_name = "계정명/데이터셋 이름"
dataset_name = "kgmyh/olympic-ragas-eval-dataset"
eval_dataset.push_to_hub(dataset_name)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/kgmyh/olympic-ragas-eval-dataset/commit/1be519557b792ca505eddbbd7e666d0370b18143', commit_message='Upload dataset', commit_description='', oid='1be519557b792ca505eddbbd7e666d0370b18143', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/kgmyh/olympic-ragas-eval-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='kgmyh/olympic-ragas-eval-dataset'), pr_revision=None, pr_num=None)

### HuggingFace hug의 평가 Datasets load

In [83]:
from datasets import load_dataset

load_eval_dataset = load_dataset(
    path=dataset_name
)

README.md:   0%|          | 0.00/360 [00:00<?, ?B/s]

c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\datasets--kgmyh--olympic-ragas-eval-dataset. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/13.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25 [00:00<?, ? examples/s]

In [85]:
load_eval_dataset["train"]

Dataset({
    features: ['user_input', 'retrieved_contexts', 'reference'],
    num_rows: 25
})